In [23]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [24]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


In [25]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [26]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [27]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

In [28]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

C:\Users\vaca1\Anaconda3\lib\site-packages\tensorflow_core\python\framework\tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
C:\Users\vaca1\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:1389: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 227846 samples, validate on 56961 samples
Epoch 1/30
227846/227846 - 6s - loss: 2.2489e-06 - fn: 48.0000 - fp: 25108.0000 - tn: 202321.0000 - tp: 369.0000 - precision: 0.0145 - recall: 0.8849 - val_loss: 1.5033e-06 - val_fn: 4.0000 - val_fp: 4643.0000 - val_tn: 52243.0000 - val_tp: 71.0000 - val_precision: 0.0151 - val_recall: 0.9467
Epoch 2/30
227846/227846 - 3s - loss: 1.6534e-06 - fn: 37.0000 - fp: 8131.0000 - tn: 219298.0000 - tp: 380.0000 - precision: 0.0446 - recall: 0.9113 - val_loss: 1.3702e-06 - val_fn: 12.0000 - val_fp: 205.0000 - val_tn: 56681.0000 - val_tp: 63.0000 - val_precision: 0.2351 - val_recall: 0.8400
Epoch 3/30
227846/227846 - 3s - loss: 1.2621e-06 - fn: 28.0000 - fp: 7480.0000 - tn: 219949.0000 - tp: 389.0000 - precision: 0.0494 - recall: 0.9329 - val_loss: 1.0130e-06 - val_fn: 10.0000 - val_fp: 373.0000 - val_tn: 56513.0000 - val_tp: 65.0000 - val_precision: 0.1484 - val_recall: 0.8667
Epoch 4/30
227846/2

Epoch 29/30
227846/227846 - 3s - loss: 7.4961e-07 - fn: 4.0000 - fp: 4304.0000 - tn: 223125.0000 - tp: 413.0000 - precision: 0.0876 - recall: 0.9904 - val_loss: 3.6293e-06 - val_fn: 10.0000 - val_fp: 333.0000 - val_tn: 56553.0000 - val_tp: 65.0000 - val_precision: 0.1633 - val_recall: 0.8667
Epoch 30/30
227846/227846 - 3s - loss: 7.4515e-07 - fn: 5.0000 - fp: 4714.0000 - tn: 222715.0000 - tp: 412.0000 - precision: 0.0804 - recall: 0.9880 - val_loss: 2.5592e-06 - val_fn: 7.0000 - val_fp: 627.0000 - val_tn: 56259.0000 - val_tp: 68.0000 - val_precision: 0.0978 - val_recall: 0.9067


Useful resources:
- https://keras.io/examples/structured_data/imbalanced_classification/